In [ ]:
# Libraries Used
import collections
from collections import OrderedDict
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import os
from google.colab import drive
import pandas as pd
import tensorflow as tf
from random import random
import keras
import random

In [ ]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# compiled_stacked_coors.csv - for every training/validation image, the 68 points were taken out (and only the points pertaining to the nose were saved)
#  - the 68 points essentially represent the facial landmarks of a person's face that are pertinent to defining facial features (e.g. the nose, the mouth, etc.)
#. - We honed in 
df = pd.read_csv('/content/gdrive/MyDrive/personality_datasets/compiled_stacked_coors.csv')
df = df.T
dictionary = df.to_dict()

# converting .csv to list format - this data ('[(90, 112), (90, 120)]') was stored as a string so this is just changing it to a list so that Python can use it
compiled_stacked_coors = []
compiled_stacked_coors_values = []

for i in range(0, len(df)-1):
  string = dictionary[0][str(i)]
  index_value = dictionary[1][str(i)]

  split_string = string.split(', (')
  split_string[0] = split_string[0][2:]
  split_string[len(split_string)-1] = split_string[len(split_string)-1][:-1]

  not_paranthesis = True
  new_string = []
  for i in range(len(split_string)):
    split_string[i] = '(' + split_string[i]
    string = split_string[i]
    string = string.split(', ')
    if (string[0] == '('):
      not_paranthesis = False
      continue
    string[0] = string[0][1:]
    string[1] = string[1][:-1]
    if (not string[0].isnumeric() or not string[1].isnumeric() or not index_value.isnumeric()):
      not_paranthesis = False
    new_string.append([int(string[0])/150.0, int(string[1])/150.0])

  if (not_paranthesis):
    if (len(new_string) == 9):
      compiled_stacked_coors.append(new_string)
      compiled_stacked_coors_values.append(int(index_value))

In [ ]:
# Randomly shuffling the dataset
c = list(zip(compiled_stacked_coors, compiled_stacked_coors_values))
random.shuffle(c)
compiled_stacked_coors, compiled_stacked_coors_values = zip(*c)

# Splitting up train and validation
train = compiled_stacked_coors[:int(0.6*len(compiled_stacked_coors))]
train_values = compiled_stacked_coors_values[:int(0.6*len(compiled_stacked_coors_values))]
valid = compiled_stacked_coors[int(0.6*len(compiled_stacked_coors)):]
valid_values = compiled_stacked_coors_values[int(0.6*len(compiled_stacked_coors_values)):]

# Converting to one-hot values so that no bias is added to the data based on big_nose being 1
train_values = tf.one_hot(train_values, depth = 2)
valid_values = tf.one_hot(valid_values, depth = 2)

# Converting to a Tensorflow dataset and splitting based on batch size
train_dataset = tf.data.Dataset.from_tensor_slices((list(train), train_values)).shuffle(len(train_values)).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((list(valid), valid_values)).shuffle(len(valid_values)).batch(64)

In [ ]:
# Making a relatively simple model - just a series of linear hidden layers that sequentially decrease in node side - 'sigmoid' was arbitrarily chosen as the activation function
model = keras.Sequential([
    keras.layers.Reshape(target_shape=(18*1,), input_shape=(9,2)),
    keras.layers.Dense(units=128, activation='sigmoid'),
    keras.layers.Dense(units=64, activation='sigmoid'),
    keras.layers.Dense(units=32, activation='sigmoid'),
    keras.layers.Dense(units=10, activation='sigmoid'),
    keras.layers.Dense(units=2, activation='sigmoid')
])

In [ ]:
# Compiling and running the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), 
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_dataset.repeat(), 
    epochs=10, 
    steps_per_epoch=500,
    batch_size = 32,
    validation_data=val_dataset.repeat(), 
    validation_steps = 2
)

Epoch 1/10
500/500 [==============================] - 2s 3ms/step - loss: 0.6969 - accuracy: 0.5007 - val_loss: 0.6854 - val_accuracy: 0.5625
Epoch 2/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6941 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.4844
Epoch 3/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6932 - accuracy: 0.5093 - val_loss: 0.6989 - val_accuracy: 0.4609
Epoch 4/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6934 - accuracy: 0.5023 - val_loss: 0.6925 - val_accuracy: 0.5234
Epoch 5/10
500/500 [==============================] - 1s 3ms/step - loss: 0.6934 - accuracy: 0.5006 - val_loss: 0.6942 - val_accuracy: 0.4766
Epoch 6/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6934 - accuracy: 0.4997 - val_loss: 0.6938 - val_accuracy: 0.4531
Epoch 7/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6936 - accuracy: 0.5011 - val_loss: 0.6911 - val_accuracy: 0.5391
Epoch 